In [1]:
# 优先队列
from queue import PriorityQueue
import math
import random

# 定义迷宫大小和墙的符号
MAZE_WIDTH = 33
MAZE_HEIGHT = 33
WALL = '1'
EMPTY = '0'


# dfs算法生成迷宫
def generate_palace(start):
    palace = [[WALL for _ in range(MAZE_WIDTH)] for _ in range(MAZE_HEIGHT)]
    visited = [[False for _ in range(MAZE_WIDTH)] for _ in range(MAZE_HEIGHT)]

    def dfs(x, y):
        visited[y][x] = True
        directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx * 2, y + dy * 2
            if 0 <= nx < MAZE_WIDTH and 0 <= ny < MAZE_HEIGHT and not visited[ny][nx]:
                palace[ny][nx] = EMPTY
                palace[y + dy][x + dx] = EMPTY
                dfs(nx, ny)

    start_x, start_y = start
    palace[start_y][start_x] = EMPTY
    dfs(start_x, start_y)

    return Palace(palace)


class Palace:
    def __init__(self, palace):
        self.palace = palace

    def print_palace(self):
        for row in self.palace:
            print(", ".join(row))

    def neighbors(self, current):
        x, y = current
        neighbors = []
        directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < MAZE_WIDTH
                and 0 <= ny < MAZE_HEIGHT
                and self.palace[nx][ny] != WALL
            ):
                neighbors.append((nx, ny))
        return neighbors

    def cost(self, current, next):
        return 1


# 预估代价
def heuristic1(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def heuristic2(goal, next, current):
    return heuristic1(goal, next) + heuristic1(goal, current)


# def heuristic3(a, b):
#     # 权重
#     w1, w2 = 0.55, 0.45
#     return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


# 借鉴此博客
# https:#www.redblobgames.com/pathfinding/a-star/introduction.html#astar
def a_star_search(graph, start, goal, mode):
    # 存放边界方块
    frontier = PriorityQueue()
    frontier.put(start, 0)
    # 从当前方块到之前方块的映射，代表路径的来向
    came_from = dict()
    # 代表方块的当前代价
    cost_so_far = dict()
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()

        if current == goal:
            break

        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                if mode == 1:
                    priority = new_cost + heuristic1(goal, next)
                elif mode == 2:
                    priority = new_cost + heuristic2(goal, next, current)
                else:
                    # priority = new_cost + heuristic3(goal, next, current)
                    pass
                frontier.put(next, priority)
                came_from[next] = current

    return came_from, cost_so_far


def reconstruct_path(came_from, start, goal):
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.reverse()  # 反转路径，使起始点到目标点的顺序
    return path


def show_puzzle(palace: Palace, path):
    # print("生成的迷宫：")
    # palace.print_palace()
    # print()
    print("A*的路径（@）:")
    for x, y in path:
        palace.palace[x][y] = "@"
    palace.print_palace()


if __name__ == "__main__":
    start = (0, 0)  # 自定义起点
    goal = (MAZE_WIDTH - 1, MAZE_HEIGHT - 1)  # 自定义目标点
    palace = generate_palace(start)
    came_from, cost_so_far = a_star_search(palace, start, goal, 2)
    path = reconstruct_path(came_from, start, goal)
    show_puzzle(palace, path)


A*的路径（@）:
@, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
@, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
@, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0
@, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0
@, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0
@, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0
@, @, @, @, @, @, @, @, @, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0
1, 1, 1, 1, 1, 1, 1, 1, @, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0
0, 0, 0, 0, 0, 0, 0, 1, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @, @
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, @
0, 1, 0, 1